In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets transformers huggingface_hub torchaudio librosa jiwer
!apt install git-lfs
!pip install accelerate -U
!pip install nvidia-smi
!pip install transformers[torch]
!pip install wandb
!pip install matplotlib seaborn
!pip install pyctcdecode
!pip install kenlm
!pip install evaluate
!sudo apt-get install git-lfs tree
import torch
import random
import pandas as pd
from datasets import load_dataset, Audio
from IPython.display import display, HTML, Audio as IPythonAudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Trainer, TrainingArguments
import re
import json
from collections import Counter
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import wandb
import os
from typing import Any, Dict, List, Union
from dataclasses import dataclass
from huggingface_hub import HfFolder

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import re
import unicodedata

# Define input and output file paths
input_path = "/content/drive/MyDrive/ASR_PROJECT_DATA_FOLDER/general_update_14th.txt"
output_path = "/content/drive/MyDrive/ASR_PROJECT_DATA_FOLDER/general_update_14th_clean2.txt"

# Read the text file
with open(input_path, "r", encoding="utf-8") as file:
    text = file.read()

# Normalize accented characters to their ASCII equivalents
normalized_text = unicodedata.normalize("NFKD", text)
ascii_text = normalized_text.encode("ASCII", "ignore").decode("utf-8")

# Convert the entire text to lowercase (if not already)
lower_text = ascii_text.lower()

# Remove numbers, punctuation (including dollar signs), and any other non-English characters.
# This regex keeps only lowercase English letters (a-z) and whitespace.
clean_text = re.sub(r'[^a-z\s]', '', lower_text)

# Write the cleaned text to a new file
with open(output_path, "w", encoding="utf-8") as file:
    file.write(clean_text)

print("Cleaned text saved to:", output_path)

# Extract and print each unique character (ignoring whitespace)
unique_chars = sorted(set(clean_text.replace(" ", "")))

# In case any character is not lowercase (should not happen due to our conversion), ensure all are lowercase.
unique_chars = [char.lower() for char in unique_chars]

print("Unique characters in the cleaned text:")
print(" ".join(unique_chars))


Cleaned text saved to: /content/drive/MyDrive/ASR_PROJECT_DATA_FOLDER/general_update_14th_clean2.txt
Unique characters in the cleaned text:
	 
 a b c d e f g h i j k l m n o p q r s t u v w x y z


In [ ]:
import re

# Path to your input and output files
input_file = "/content/drive/MyDrive/ASR AFRICA/Copy of Amharic general latest.txt"
output_file = "/content/drive/MyDrive/ASR AFRICA/Filtered_Amharic_Text.txt"

# Define the Amharic alphabet
alphabet = 'ሀሁሂሃሄህሆለሉሊላሌልሎሏሐሑሒሓሔሕሖሗመሙሚማሜምሞሟሠሡሢሣሤሥሦሧረሩሪራሬርሮሯሰሱሲሳሴስሶሷሸሹሺሻሼሽሾሿቀቁቂቃቄቅቆቋበቡቢባቤብቦቧቨቩቪቫቬቭቮቯተቱቲታቴትቶቷቸቹቺቻቼችቾቿኀኁኂኃኄኅኆኋነኑኒናኔንኖኗኘኙኚኛኜኝኞኟአኡኢኣኤእኦከኩኪካኬክኮኳኸኹኺኻኼኽኾዃወዉዊዋዌውዎዐዑዒዓዔዕዖዘዙዚዛዜዝዞዟዠዡዢዣዤዥዦዧየዩዪያዬይዮደዱዲዳዴድዶዷጀጁጂጃጄጅጆጇገጉጊጋጌግጎጓጠጡጢጣጤጥጦጧጨጩጪጫጬጭጮጯጰጱጲጳጴጵጶጷጸጹጺጻጼጽጾጿፀፁፂፃፄፅፆፈፉፊፋፌፍፎፏፐፑፒፓፔፕፖፗ '

# Define normalization functions
def normalize_char_level_missmatch(input_token):
    rep1 = re.sub('[ሃኅኃሐሓኻ]', 'ሀ', input_token)
    rep2 = re.sub('[ሑኁዅ]', 'ሁ', rep1)
    rep3 = re.sub('[ኂሒኺ]', 'ሂ', rep2)
    rep4 = re.sub('[ኌሔዄ]', 'ሄ', rep3)
    rep5 = re.sub('[ሕኅ]', 'ህ', rep4)
    rep6 = re.sub('[ኆሖኾ]', 'ሆ', rep5)
    rep7 = re.sub('[ሠ]', 'ሰ', rep6)
    rep8 = re.sub('[ሡ]', 'ሱ', rep7)
    rep9 = re.sub('[ሢ]', 'ሲ', rep8)
    rep10 = re.sub('[ሣ]', 'ሳ', rep9)
    rep11 = re.sub('[ሤ]', 'ሴ', rep10)
    rep12 = re.sub('[ሥ]', 'ስ', rep11)
    rep13 = re.sub('[ሦ]', 'ሶ', rep12)
    rep14 = re.sub('[ዓኣዐ]', 'አ', rep13)
    rep15 = re.sub('[ዑ]', 'ኡ', rep14)
    rep16 = re.sub('[ዒ]', 'ኢ', rep15)
    rep17 = re.sub('[ዔ]', 'ኤ', rep16)
    rep18 = re.sub('[ዕ]', 'እ', rep17)
    rep19 = re.sub('[ዖ]', 'ኦ', rep18)
    rep20 = re.sub('[ጸ]', 'ፀ', rep19)
    rep21 = re.sub('[ጹ]', 'ፁ', rep20)
    rep22 = re.sub('[ጺ]', 'ፂ', rep21)
    rep23 = re.sub('[ጻ]', 'ፃ', rep22)
    rep24 = re.sub('[ጼ]', 'ፄ', rep23)
    rep25 = re.sub('[ጽ]', 'ፅ', rep24)
    rep26 = re.sub('[ጾ]', 'ፆ', rep25)
    rep27 = re.sub('(ሉ[ዋአ])', 'ሏ', rep26)
    rep28 = re.sub('(ሙ[ዋአ])', 'ሟ', rep27)
    rep29 = re.sub('(ቱ[ዋአ])', 'ቷ', rep28)
    rep30 = re.sub('(ሩ[ዋአ])', 'ሯ', rep29)
    rep31 = re.sub('(ሱ[ዋአ])', 'ሷ', rep30)
    rep32 = re.sub('(ሹ[ዋአ])', 'ሿ', rep31)
    rep33 = re.sub('(ቁ[ዋአ])', 'ቋ', rep32)
    rep34 = re.sub('(ቡ[ዋአ])', 'ቧ', rep33)
    rep35 = re.sub('(ቹ[ዋአ])', 'ቿ', rep34)
    rep36 = re.sub('(ሁ[ዋአ])', 'ኋ', rep35)
    rep37 = re.sub('(ኑ[ዋአ])', 'ኗ', rep36)
    rep38 = re.sub('(ኙ[ዋአ])', 'ኟ', rep37)
    rep39 = re.sub('(ኩ[ዋአ])', 'ኳ', rep38)
    rep40 = re.sub('(ዙ[ዋአ])', 'ዟ', rep39)
    rep41 = re.sub('(ጉ[ዋአ])', 'ጓ', rep40)
    rep42 = re.sub('(ደ[ዋአ])', 'ዷ', rep41)
    rep43 = re.sub('(ጡ[ዋአ])', 'ጧ', rep42)
    rep44 = re.sub('(ጩ[ዋአ])', 'ጯ', rep43)
    rep45 = re.sub('(ጹ[ዋአ])', 'ጿ', rep44)
    rep46 = re.sub('(ፉ[ዋአ])', 'ፏ', rep45)
    rep47 = re.sub('[ቊ]', 'ቁ', rep46)
    rep48 = re.sub('[ኵ]', 'ኩ', rep47)
    return rep48

def remove_punc_and_special_chars(text):
    text = re.sub('[።፣፤፥፦፧፨፠]', '', text)
    text = re.sub('[\!\@\#\$\%\^\&\*\(\)\[\]\{\}\;\:\,\.\?\'\"\\\/\`\~\-\+\=]', '', text)
    return text

def remove_ascii_and_numbers(text_input):
    rm_num_and_ascii = re.sub('[A-Za-z0-9]', '', text_input)
    return rm_num_and_ascii

# Combine preprocessing steps
def preprocess_sentence(sentence):
    sentence = normalize_char_level_missmatch(sentence)
    sentence = remove_punc_and_special_chars(sentence)
    sentence = remove_ascii_and_numbers(sentence)
    sentence = ' '.join(sentence.split()).strip()
    return sentence if all(char in alphabet for char in sentence) else None

# Read input file, process each line, and save valid sentences
with open(input_file, "r", encoding="utf-8") as infile, open(output_file, "w", encoding="utf-8") as outfile:
    for line in infile:
        normalized_sentence = preprocess_sentence(line.strip())
        if normalized_sentence:  # Save only valid sentences
            outfile.write(normalized_sentence + "\n")
print(f"Filtered dataset saved to {output_file}")


In [ ]:
import re

# Define a function to extract unique characters from a file
def extract_unique_characters(file_path):
    unique_chars = set()  # Use a set to store unique characters
    with open(file_path, "r", encoding="utf-8") as infile:
        for line in infile:
            unique_chars.update(line.strip())  # Add characters from each line to the set
    return unique_chars, len(unique_chars)

# Paths to input and filtered output files
input_file = "/content/drive/MyDrive/ASR AFRICA/LUO_General_clean.txt"
output_file = "/content/drive/MyDrive/ASR AFRICA/LUO_General_clean_output.txt"

# Define the Amharic alphabet
alphabet = "abcdefghijklmnopqrstuvwxyz "
# Reuse the preprocessing function from the previous script
def preprocess_sentence(sentence):
    # sentence = normalize_char_level_missmatch(sentence)
    sentence = remove_punc_and_special_chars(sentence)
    # sentence = remove_ascii_and_numbers(sentence)
    sentence = ' '.join(sentence.split()).strip()
    return sentence if all(char in alphabet for char in sentence) else None

# Read and preprocess the input file
with open(input_file, "r", encoding="utf-8") as infile, open(output_file, "w", encoding="utf-8") as outfile:
    for line in infile:
        normalized_sentence = preprocess_sentence(line.strip())
        if normalized_sentence:  # Save only valid sentences
            outfile.write(normalized_sentence + "\n")

# Extract unique characters from the filtered output
unique_chars, char_count = extract_unique_characters(output_file)

# Print the results
print(f"Unique characters in the filtered dataset: {''.join(sorted(unique_chars))}")
print(f"Number of unique characters: {char_count}")


Unique characters in the filtered dataset:  abcdefghijklmnopqrstuvwxyz
Number of unique characters: 27


In [ ]:
# Path to the filtered output file
output_file = "/content/drive/MyDrive/ASR_PROJECT_DATA_FOLDER/general_update_14th_clean2.txt"

# Function to count the number of sentences in the file
def count_sentences(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        sentences = f.readlines()  # Read all lines
    return len(sentences)

# Count the sentences
num_sentences = count_sentences(output_file)

# Print the result
print(f"Number of sentences in the file: {num_sentences}")


Number of sentences in the file: 44297


In [ ]:
# Step 1: Install the necessary dependencies for KenLM
!sudo apt install build-essential cmake libboost-system-dev libboost-thread-dev libboost-program-options-dev libboost-test-dev libeigen3-dev zlib1g-dev libbz2-dev liblzma-dev
# Step 2: Download and build KenLM
!wget -O - https://kheafield.com/code/kenlm.tar.gz | tar xz
!mkdir kenlm/build && cd kenlm/build && cmake .. && make -j2
!ls kenlm/build/bin

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
libboost-program-options-dev is already the newest version (1.74.0.3ubuntu7).
libboost-program-options-dev set to manually installed.
libboost-system-dev is already the newest version (1.74.0.3ubuntu7).
libboost-system-dev set to manually installed.
libboost-thread-dev is already the newest version (1.74.0.3ubuntu7).
libboost-thread-dev set to manually installed.
libbz2-dev is already the newest version (1.0.8-5build1).
libbz2-dev set to manually installed.
liblzma-dev is already the newest version (5.2.5-2ubuntu1).
liblzma-dev set to manually installed.
libboost-test-dev is already the newest version (1.74.0.3ubuntu7).
libboost-test-dev set to manually installed.
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-2ubuntu9.2).
zlib1g-dev set to manually installed.

In [ ]:
import os
# Step 3: Path to your preprocessed Amharic text file (from your previous steps)
input_file_path ="/content/drive/MyDrive/ASR_PROJECT_DATA_FOLDER/general_update_14th_clean2.txt"
output_lm_file = "text.txt"
# Step 4: Write the text to be used for language modeling into a file
with open(input_file_path, 'r', encoding='utf-8') as amharic_file:
    lines = amharic_file.readlines()
# Step 5: Write the cleaned and filtered text data into the text.txt file
with open(output_lm_file, 'w', encoding='utf-8') as file:
    file.write(" ".join([line.strip() for line in lines]))
print(f"Text data saved to {output_lm_file}")

Text data saved to text.txt


In [ ]:
# Step 6: Use KenLM to build a 5-gram language model from the preprocessed Amharic text
!kenlm/build/bin/lmplz -o 5 < "text.txt" > "5gorm.arpa" --discount_fallback

# Step 7: Check the contents of the generated 5-gram ARPA file
!head -20 5gorm.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /content/text.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 1301150 types 92365
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:1108380 2:1062090688 3:1991420160 4:3186272000 5:4646646784
Statistics:
1 92364 D1=0.707054 D2=1.01098 D3+=1.37006
2 609737 D1=0.84858 D2=1.17718 D3+=1.37873
3 906669 D1=0.946357 D2=1.40655 D3+=1.55095
4 996777 D1=0.979167 D2=1.56792 D3+=1.77509
5 1043347 D1=0.931291 D2=1.73552 D3+=1.99962
Memory estimate for binary LM:
type    MB
probing 77 assuming -p 1.5
probing 92 assuming -r models -p 1.5
trie    38 without quantization
trie    21 assuming -q 8 -b 8 quantization 
trie    34 assuming -a 22 array pointer compression
trie    17 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and s

In [ ]:
# Step 8: Modify the ARPA file to include the end-of-sentence token

with open("5gorm.arpa", "r") as read_file, open("5gram_corrected_orm.arpa", "w") as write_file:
    has_added_eos = False
    for line in read_file:
        if not has_added_eos and "ngram 1=" in line:
            count = line.strip().split("=")[-1]
            write_file.write(line.replace(f"{count}", f"{int(count)+1}"))
        elif not has_added_eos and "<s>" in line:
            write_file.write(line)
            write_file.write(line.replace("<s>", "</s>"))
            has_added_eos = True
        else:
            write_file.write(line)
print("Corrected ARPA file saved as 5gram_corrected_orm.arpa")

Corrected ARPA file saved as 5gram_corrected_orm.arpa


In [ ]:
!head -20 5gram_corrected_orm.arpa

\data\
ngram 1=92365
ngram 2=609737
ngram 3=906669
ngram 4=996777
ngram 5=1043347

\1-grams:
-5.825711	<unk>	0
0	<s>	-0.071307056
0	</s>	-0.071307056
-3.5382977	bataskaanni	-1.2350684
-4.762271	kaatolikii	-0.15322779
-2.692583	itoophiyaa	-0.39712065
-4.566587	vaatikaan	-0.19026749
-2.592982	keessatti	-0.3108478
-3.437733	qabeenya	-0.2454331
-2.8443568	waggaa	-0.57766914
-3.5432458	akkamiin	-0.15603077
-5.380179	horatte	-0.071307056


In [ ]:
!kenlm/build/bin/query 5gram_corrected_luo.arpa <test_text.txt

/bin/bash: line 1: test_text.txt: No such file or directory


In [ ]:
# # Step 9: Convert the corrected ARPA file to a binary format using KenLM's build_binary tool
# os.system("kenlm/build/bin/build_binary 5gram_corrected.arpa 5gram.binary")
# print("Binary language model saved as 5gram.binary")

In [ ]:
# # Step 10: Move the binary model to Google Drive
# import os
# os.system("mv /content/5gram.binary  /content/drive/MyDrive/ASR AFRICA/Extracted1/5gram.binary")
# print("Binary language model moved to Google Drive at /content/drive/MyDrive/ASR AFRICA/Extracted1/5gram.binary")

In [ ]:
# Move the ARPA file to Google Drive
!mv /content/5gram_corrected_orm.arpa /content/drive/MyDrive/DATASETS/5gram_corrected_orm.arpa